## EECS 731 Project 4: Regression
### by Matthew Taylor

### Import required modules

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

### Load datasets

In [2]:
initial_elos = pd.read_csv('data/initial_elos.csv')
nfl_games = pd.read_csv('data/nfl_games.csv')

In [3]:
# The 'initial_elos' datasets contains team abbreviation encodings through integer indices

initial_elos.head()

,team,elo
0,RII,1503.947
1,STP,1300.000
2,BFF,1478.004
3,WBU,1300.000
4,RCH,1503.420


In [4]:
# A majority of the data is already numerical, which simplifies data preparation

nfl_games.head()

,date,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1
0,1920-09-26,1920,0,0,RII,STP,1503.947,1300.000,0.824651,48,0,1.0
1,1920-10-03,1920,0,0,AKR,WHE,1503.420,1300.000,0.824212,43,0,1.0
2,1920-10-03,1920,0,0,RCH,ABU,1503.420,1300.000,0.824212,10,0,1.0
3,1920-10-03,1920,0,0,DAY,COL,1493.002,1504.908,0.575819,14,0,1.0
4,1920-10-03,1920,0,0,RII,MUN,1516.108,1478.004,0.644171,45,0,1.0


### Data Preparation

In [5]:
years = []
months = []
days = []

team1_ids = []
team2_ids = []

for index, row in nfl_games.iterrows():
    # Break date column up into three separate columns
    date = row.date.split('-')
    
    years.append(date[0])
    months.append(date[1])
    days.append(date[2])
    
    # Replace team abbreviations with integer IDs
    team1_ids.append(initial_elos.index[initial_elos['team'] == row['team1']].values[0])
    team2_ids.append(initial_elos.index[initial_elos['team'] == row['team2']].values[0])
    
# Add new columns to the dataframe
nfl_games.insert(1, 'year', years)
nfl_games.insert(2, 'month', months)
nfl_games.insert(3, 'day', days)
nfl_games.insert(9, 'team1_id', team1_ids)
nfl_games.insert(10, 'team2_id', team2_ids)

# Remove unwanted columns
nfl_games = nfl_games.drop(columns=['date', 'team1', 'team2'])

# Inspect resulting dataframe to verify data preparation steps
nfl_games.head()

,year,month,day,season,neutral,playoff,team1_id,team2_id,elo1,elo2,elo_prob1,score1,score2,result1
0,1920,09,26,1920,0,0,0,1,1503.947,1300.000,0.824651,48,0,1.0
1,1920,10,03,1920,0,0,13,14,1503.420,1300.000,0.824212,43,0,1.0
2,1920,10,03,1920,0,0,4,5,1503.420,1300.000,0.824212,10,0,1.0
3,1920,10,03,1920,0,0,6,7,1493.002,1504.908,0.575819,14,0,1.0
4,1920,10,03,1920,0,0,0,8,1516.108,1478.004,0.644171,45,0,1.0


### Train-Test Split

In [6]:
# Separate the data into two types, training and testing (80/20)
# two outputs are required to predict the score of each team
# since each regression model can only produce one result

input_columns = nfl_games[['year', 'month', 'day', 'season', 'neutral', 'playoff', 'team1_id', 'team2_id', 'elo1', 'elo2', 'elo_prob1', 'result1']].values
output_columns1 = nfl_games['score1'].values
output_columns2 = nfl_games['score2'].values

train_input,   test_input   = train_test_split(input_columns, test_size=0.2, random_state=0)
train_output1, test_output1 = train_test_split(output_columns1, test_size=0.2, random_state=0)
train_output2, test_output2 = train_test_split(output_columns2, test_size=0.2, random_state=0)

### Linear Regression

In [7]:
# Train linear regression model

lr1 = LinearRegression().fit(train_input, train_output1)
lr2 = LinearRegression().fit(train_input, train_output2)

In [8]:
# Score linear regression model

lr1_score = round(lr1.score(test_input, test_output1) * 100, 2)
lr2_score = round(lr2.score(test_input, test_output2) * 100, 2)
lr_average = round((lr1_score + lr2_score) / 2, 2)

print('Team 1 scores correctly predicted {}% of the time'.format(lr1_score))
print('Team 2 scores correctly predicted {}% of the time\n'.format(lr2_score))
print('Scores correctly predicted an average of {}% of the time'.format(lr_average))

Team 1 scores correctly predicted 34.29% of the time
Team 2 scores correctly predicted 37.69% of the time

Scores correctly predicted an average of 35.99% of the time


### Random Forest Regressor

In [ ]:
# Train random forest

rf1 = RandomForestRegressor(n_estimators=100, random_state=0).fit(train_input, train_output1)
rf2 = RandomForestRegressor(n_estimators=100, random_state=0).fit(train_input, train_output2)

In [ ]:
# Test random forest

rf1_score = round(rf1.score(test_input, test_output1) * 100, 2)
rf2_score = round(rf2.score(test_input, test_output2) * 100, 2)
rf_average = round((rf1_score + rf2_score) / 2, 2)

print('Team 1 scores correctly predicted {}% of the time'.format(rf1_score))
print('Team 2 scores correctly predicted {}% of the time\n'.format(rf2_score))
print('Scores correctly predicted an average of {}% of the time'.format(rf_average))